In [9]:
import pandas as pd 
import xmltodict
from urllib.request import urlopen 
import json


In [10]:
url = "https://itunes.apple.com/kr/rss/customerreviews/id=1464496236/sortBy=mostRecent/xml"
response = urlopen(url).read() 
response


b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="ko">\n\t\t<id>https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/id=1464496236/sortBy=mostRecent/xml</id><title>iTunes Store:\xec\x82\xac\xec\x9a\xa9\xec\x9e\x90 \xed\x9b\x84\xea\xb8\xb0</title><updated>2022-05-19T23:45:27-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=kr&amp;id=29141"/><link rel="self" href="https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/id=1464496236/sortBy=mostRecent/xml"/><link rel="first" href="https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1464496236/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1464496236/sortBy=mostRecent/xml"/><link rel="last" href="https://itunes.apple.com/kr/rss/customerreviews/page=10/id=1464496236/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1464496236/sortBy=mostRecent/xml"/

In [12]:
#parsing xml
xml = xmltodict.parse(response) 
XmlToJson = json.loads(json.dumps(xml))



In [88]:
#1464496236 카카오페이
#1554807824 네이버페이
res_dict = [] 
for j in range(1,11):
    url = "https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1464496236/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1464496236/sortBy=mostRecent/xml"% j
    response = urlopen(url).read() 
    
    xml = xmltodict.parse(response) 
    XmlToJson = json.loads(json.dumps(xml))
    
    for i in range(len(XmlToJson['feed']['entry'])): 
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'], 
            'STAR' : XmlToJson['feed']['entry'][i]['im:rating'], 
#             'LIKE' : XmlToJson['feed']['entry'][i]['im:voteSum'], 
#             'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']), 
#             'TITLE' : XmlToJson['feed']['entry'][i]['title'], 
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text'] 
        })
res_df = pd.DataFrame(res_dict) 
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00") 
# res_df.to_csv('../data/IOS_ReviewData.v1.0.csv')



In [89]:
app_kakao=res_df.sort_values(['DATE'],ascending=False)

In [91]:
app_kakao.to_csv('카카오페이_앱스토어.csv',index=False,encoding='utf-8-sig')

In [87]:
app_naver.to_csv('네이버페이_앱스토어.csv',index=False,encoding='utf-8-sig')

In [90]:
app_kakao

,DATE,STAR,REVIEW
0,2022-05-18 19:31:40,4,"차트 내 구매평균이 어떤 종목에서는 보이고, 어떤 종목은 보이지 않네요!"
1,2022-05-18 06:00:30,1,카카오페이에서 송금하려고 하는데 미성년자라 민증이 안나온다는 이유로 카카오페이에 제...
2,2022-05-17 11:00:26,4,카드 사용액 조회 화면에 있는 또래 평균과 내 사용금액을 비교하는 그래프에서 또래 ...
3,2022-05-17 02:11:04,4,타증권으로 주식 옮기는거 만들어주세요
4,2022-05-14 23:45:26,1,오류 개많음 진짜 사람 개빡치게 함
...,...,...,...
495,2021-01-27 11:03:32,3,결제도 빨리되고 해서 좋은데 왜 충전을 할려면 만원 이상만 할수있는거예요? 좀 내려...
496,2021-01-27 02:58:01,3,민증 인증만 지금 몇시간째 못하고있어요...ㅠ 죽어도안돼요 진짜 ㅠ최근에 다른 앱에...
497,2021-01-26 23:53:35,4,금융캘린더 위젯사용할려고 카드등록 다했는ㄷㅔ도 캘린더에 계속 카드등록하라고 나와요...
498,2021-01-26 18:10:12,5,앱 재미있게 활용하고있습니다


In [3]:
import pandas as pd
df_n_p=pd.read_csv('네이버페이_플레이스토어.csv')
df_n_a=pd.read_csv('네이버페이_앱스토어.csv')
naver=pd.merge(df_n_p,df_n_a,how='outer')
naver=naver.sort_values(['DATE'],ascending=False)
naver.reset_index(drop=True,inplace=True)


In [6]:
naver.STAR.value_counts()

5    301
1    187
4     90
3     59
2     25
Name: STAR, dtype: int64